# ImgLib2 

Example 03: Writing generic algorithms

**Imglib2 Examples 01 and 02** tried to introduce important tools you need in order to implement algorithms with ImgLib2. This example will show three generic implementations of algorithms computing the min/max, average, as well as the center of mass.

The core idea is to implement algorithms as generic as possible in order to maximize code-*reusability.* In general, a good way to start is to think: What are the minimal requirements in order to implement algorithm X? This applies to all of the following three concepts:

* **Type**: You should always use the most abstract [Type](http://javadoc.imagej.net/ImgLib2/net/imglib2/type/Type.html) possible, i.e. the one that just offers enough operations to perform your goal. In this way, the algorithm will be able to run on [Type](http://javadoc.imagej.net/ImgLib2/net/imglib2/type/Type.html)s you might not even have thought about when implementing it. A good example is the min&max search in example 3a. Instead of implementing it for [FloatType](http://javadoc.imagej.net/ImgLib2/net/imglib2/type/numeric/real/FloatType.html) or the more abstract [RealType](http://javadoc.imagej.net/ImgLib2/net/imglib2/type/numeric/RealType.html), we implement it for the even more abstract [Comparable](https://docs.oracle.com/javase/7/docs/api/java/lang/Comparable.html) & [Type](http://javadoc.imagej.net/ImgLib2/net/imglib2/type/Type.html).
* **Image data**: Every algorithm should only demand those interfaces that it requires, not specific implementations of it like [Img](http://javadoc.imagej.net/ImgLib2/net/imglib2/img/Img.html). You might require [RandomAccessible](http://javadoc.imagej.net/ImgLib2/net/imglib2/RandomAccessible.html) (infinite), [RandomAccessibleInterval](http://javadoc.imagej.net/ImgLib2/net/imglib2/RandomAccessibleInterval.html) (finite), [Iterable](http://javadoc.imagej.net/Java/java/lang/Iterable.html?is-external=true) (values without location), [IterableInterval](http://javadoc.imagej.net/ImgLib2/net/imglib2/IterableInterval.html) (values and their location) or their corresponding interfaces for real-valued locations [RealRandomAccessible](http://javadoc.imagej.net/ImgLib2/net/imglib2/RealRandomAccessible.html), [RealRandomAccessibleRealInterval](http://javadoc.imagej.net/ImgLib2/net/imglib2/RealRandomAccessibleRealInterval.html) and [IterableRealInterval](http://javadoc.imagej.net/ImgLib2/net/imglib2/IterableRealInterval.html). Note that you can concatenate them if you need more than one property.
* **Dimensionality**: Usually there is no reason to restrict an algorithm to a certain dimensionality (like only for two-dimensional images), at least we could not really come up with a convincing example. *If the application or plugin you are developing addresses a certain dimensionality (e.g. stitching of panorama photos) it is understandable that you do not want to implement everything n-dimensionally. But try to implement as many as possible of the smaller algorithm you are using as generic, n-dimensional methods. For example, everything that requires only to **iterate** the data is usually inherently n-dimensional.*

Following those ideas, your newly-implemented algorithm will be applicable to any kind of data and dimensionality it is defined for, not only a very small domain you are currently working with. Also note that quite often this actually makes the implementation simpler.

## Min/Max search

Searching for the minimal and maximal value in a dataset is a very nice example to illustrate generic algorithms. In order to find min/max values, [Type](http://javadoc.imagej.net/ImgLib2/net/imglib2/type/Type.html)s only need to be able to compare themselves. Therefore we do not need any numeric values, we only require them to implement the (Java) interface [Comparable](https://docs.oracle.com/javase/7/docs/api/java/lang/Comparable.html). Additionally, no random access to the data is required, we simply need to iterate all pixels, also their location is irrelevant. The image data we need only needs to be [Iterable](http://javadoc.imagej.net/Java/java/lang/Iterable.html?is-external=true).

Below we show **three** small variations of the min/max search. 
* First, in Example 3a, we show the implementation as described above. 
* Second, in Example 3b, we illustrate that this also works on a standard Java ArrayList. 
* Third, in Example 3c, we show how the implementation changes if we do not only want the min/max value, but also their location. This requires to use [IterableInterval](http://javadoc.imagej.net/ImgLib2/net/imglib2/IterableInterval.html) instead, as [Cursor](http://javadoc.imagej.net/ImgLib2/net/imglib2/Cursor.html) can return their location.

### Example 3a

In [2]:
import net.imglib2.type.Type

construct01 = new Object() {   
  /**
   * Compute the min and max for any {@link Iterable}, like an {@link Img}.
   *
   * The only functionality we need for that is to iterate. Therefore we need no {@link Cursor}
   * that can localize itself, neither do we need a {@link RandomAccess}. So we simply use the
   * most simple interface in the hierarchy.
   *
   * @param input - the input that has to just be {@link Iterable}
   * @param min - the type that will have min
   * @param max - the type that will have max
   */
  public <T extends Comparable< T> & Type<T>> void computeMinMax(Iterable<T> input, T min, T max) {
    // create a cursor for the image (the order does not matter)
    Iterator<T> iterator = input.iterator()
    
    // initialize min and max with the first image value
    T type = iterator.next()
    
    min.set(type)
    max.set(type)
    
    // loop over the rest of the data and determine min and max value
    while (iterator.hasNext()) {
      // we need this type more than once
      type = iterator.next()
      
      if (type.compareTo(min) < 0) {
        min.set( type )
      }
      
      if (type.compareTo(max) > 0) {
        max.set(type)
      }
    }
  }
}

Script2​$1​@4eb32bb3

In [4]:
import io.scif.img.ImgOpener
import net.imglib2.type.numeric.real.FloatType

/**
 * Perform a generic min & max search
 *
 * @author Stephan Preibisch & Stephan Saalfeld
 *
 */

// open with ImgOpener
img = new ImgOpener().openImgs("http://samples.fiji.sc/tutorials/DrosophilaWing.tif", new FloatType()).get(0)

// create two empty variables
min = img.firstElement().createVariable()
max = img.firstElement().createVariable()

// compute min and max of the Image
construct01.computeMinMax(img, min, max)

["minimum Value (img)": min,
"maximum Value (img)": max]

Key,Value
minimum Value (img),42​.0
maximum Value (img),255​.0


### Example 3b

Note that this example works just the same way if the input is not an [Img](http://javadoc.imagej.net/ImgLib2/net/imglib2/img/Img.html), but for example just a standard Java [ArrayList](https://docs.oracle.com/javase/7/docs/api/java/util/ArrayList.html).

In [5]:
import java.util.ArrayList
import net.imglib2.type.numeric.real.FloatType
 
/**
 * Perform a generic min & max search
 *
 * @author Stephan Preibisch & Stephan Saalfeld
 *
 */

// it will work as well on a normal ArrayList
ArrayList<FloatType> list = new ArrayList<FloatType>()

// put values 0 to 10 into the ArrayList
for (int i = 0; i <= 10; ++i) {
  list.add(new FloatType(i))
}

// create two empty variables
min = new FloatType()
max = new FloatType()
 
// compute min and max of the ArrayList
construct01.computeMinMax(list, min, max)

["minimum Value (arraylist)": min,
"maximum Value (arraylist)": max]

Key,Value
minimum Value (arraylist),0​.0
maximum Value (arraylist),10​.0


### Example 3c

If we want to compute the location of the minimal and maximal pixel value, an [Iterator](http://javadoc.imagej.net/ImgLib2/net/imglib2/Iterator.html) will not be sufficient as we need location information. Instead, the location search will demand an [IterableInterval](http://javadoc.imagej.net/ImgLib2/net/imglib2/IterableInterval.html) as input data which can create [Cursor](http://javadoc.imagej.net/ImgLib2/net/imglib2/Cursor.html)s. Apart from that, the algorithm looks quite similar. Note that we do not use a **LocalizingCursor** but only a [Cursor](http://javadoc.imagej.net/ImgLib2/net/imglib2/Cursor.html); the location happens only when a new maximal or minimal value has been found while iterating the data.

In [6]:
import net.imglib2.type.Type
import net.imglib2.IterableInterval
import net.imglib2.Point
import net.imglib2.Cursor
 
/**
 * Perform a generic min/max search.
 *
 * @author Stephan Preibisch & Stephan Saalfeld
 *
 */
construct02 = new Object() {
  /**
   * Compute the location of the minimal and maximal intensity for any IterableInterval,
   * like an {@link Img}.
   *
   * The functionality we need is to iterate and retrieve the location. Therefore we need a
   * Cursor that can localize itself.
   * Note that we do not use a LocalizingCursor as localization just happens from time to time.
   *
   * @param input - the input that has to just be {@link IterableInterval}
   * @param minLocation - the location for the minimal value
   * @param maxLocation - the location of the maximal value
   */
  public <T extends Comparable<T> & Type<T>> void computeMinMaxLocation(IterableInterval<T> input, Point
    minLocation, Point maxLocation)
  {
    // create a cursor for the image (the order does not matter)
    final Cursor<T> cursor = input.cursor()
    
    // initialize min and max with the first image value
    T type = cursor.next()
    T min = type.copy()
    T max = type.copy()
    
    // loop over the rest of the data and determine min and max value
    while (cursor.hasNext()) {
      // we need this type more than once
      type = cursor.next()
      
      if (type.compareTo(min) < 0) {
        min.set(type)
        minLocation.setPosition(cursor)
      }
      
      if (type.compareTo(max) > 0) {
        max.set(type)
        maxLocation.setPosition(cursor)
      }
    }
  }
}

Script5​$1​@59650560

In [7]:
import io.scif.img.ImgOpener
import net.imglib2.type.numeric.real.FloatType
import net.imglib2.Point

// open with ImgOpener
img = new ImgOpener().openImgs("http://samples.fiji.sc/tutorials/DrosophilaWing.tif", new FloatType()).get(0)
 
 // create two location objects
 locationMin = new Point(img.numDimensions())
 locationMax = new Point(img.numDimensions())
 
 // compute location of min and max
 construct02.computeMinMaxLocation(img, locationMin, locationMax)
 
["location of minimum Value (img)": locationMin,
"location of maximum Value (img)": locationMax]

Key,Value
location of minimum Value (img),"(545,25)"
location of maximum Value (img),"(51,0)"


## Computing average

In a very similar way, one can compute the average intensity for image data. Note that we restrict the [Type](http://javadoc.imagej.net/ImgLib2/net/imglib2/type/Type.html) of data to [RealType](http://javadoc.imagej.net/ImgLib2/net/imglib2/type/numeric/RealType.html). In theory, we could use [NumericType](http://javadoc.imagej.net/ImgLib2/net/imglib2/type/numeric/NumericType.html) as it offers the possibility to add up values. However, we cannot ensure that [NumericType](http://javadoc.imagej.net/ImgLib2/net/imglib2/type/numeric/NumericType.html) provided is capable of adding up millions of pixels without overflow. And even if we would ask for a second [NumericType](http://javadoc.imagej.net/ImgLib2/net/imglib2/type/numeric/NumericType.html) that is capable of adding values up, it might still have numerical instabilities. Note that actually every Java native type has those instabilities. Therefore we use the [RealSum](http://javadoc.imagej.net/ImgLib2/net/imglib2/util/RealSum.html) class that offers correct addition of even very large amounts of pixels. As this implementation is only available for double values, we restrict the method here to [RealType](http://javadoc.imagej.net/ImgLib2/net/imglib2/type/numeric/RealType.html).

In [8]:
import net.imglib2.type.numeric.RealType
import net.imglib2.util.RealSum
 
/**
 * Perform a generic computation of average intensity
 *
 * @author Stephan Preibisch & Stephan Saalfeld
 *
 */
construct03 = new Object() {
  /**
   * Compute the average intensity for an {@link Iterable}.
   *
   * @param input - the input data
   * @return - the average as double
   */
  public <T extends RealType<T>> double computeAverage(Iterable<T> input) {
    // Count all values using the RealSum class.
    // It prevents numerical instabilities when adding up millions of pixels
    realSum = new RealSum()
    count = 0
    
    for (T type : input) {
      realSum.add(type.getRealDouble())
      ++count
    }
    
    return realSum.getSum() / count
  }
}

Script7​$1​@13d880a9

In [9]:
import io.scif.img.ImgOpener
import net.imglib2.type.numeric.real.FloatType

// open with ImgOpener
img = new ImgOpener().openImgs("http://samples.fiji.sc/tutorials/DrosophilaWing.tif", new FloatType()).get(0)
 
// compute average of the image
avg = construct03.computeAverage(img)
"average Value: " + avg

average Value: 233.45769361413045